In [27]:
import requests
from lxml import html

In [29]:
web_page = "https://coinmarketcap.com/"

resp = requests.get( web_page, headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
} )

resp

<Response [200]>

In [35]:
tree = html.fromstring(html=resp.content)

currencies = tree.xpath("//table[@id='currencies']/tbody/tr")

all_currency =  []

for currency in currencies:
    c = {
        'name': currency.xpath(".//td[contains( @class, 'currency-name')]/a/text()")[0],
        'marketcap': currency.xpath(".//td[contains( @class, 'market-cap')]/@data-usd")[0],
        'price' : currency.xpath(".//td[4]/a[@class='price']/@data-usd")[0],
        'change(24h)': currency.xpath(".//td[contains(@class, 'percent-change')]/@data-percentusd")[0]
    }
    all_currency.append(c)
    
all_currency[:3]

[{'name': 'Bitcoin',
  'marketcap': '2.17894989776e+11',
  'price': '12199.6721732',
  'change(24h)': '4.23297'},
 {'name': 'Ethereum',
  'marketcap': '25136110459.0',
  'price': '234.466167464',
  'change(24h)': '1.9222'},
 {'name': 'XRP',
  'marketcap': '13819689085.8',
  'price': '0.322342807204',
  'change(24h)': '-0.0212043'}]

In [45]:
# scapring multiple pages

from urllib.parse import urljoin

def get(list_elements):
    try:
        return list_elements.pop(0)
    except:
        return ''


all_currenceis = []


In [46]:

def scrape(url):
    resp = requests.get(url=url, headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
    })
 
    tree = html.fromstring(html=resp.content)
 
    currencies = tree.xpath("//table[@id='currencies']/tbody/tr")
    for currency in currencies:
        c = {
            "_id": int(get(currency.xpath(".//td[1]/text()"))),
            'name': get(currency.xpath(".//td[contains(@class, 'currency-name')]/a/text()")),
            'market cap': get(currency.xpath(".//td[contains(@class, 'market-cap')]/@data-usd")),
            'price': get(currency.xpath(".//td[4]/a[@class='price']/@data-usd")),
            'change(24h)': get(currency.xpath(".//td[contains(@class, 'percent-change')]/@data-percentusd"))
        }
 
        all_currenceis.append(c)
    
    next_page = tree.xpath("//ul[contains(@class, 'bottom-paginator')]/li/a[contains(text(), 'Next')]/@href")
 
    if len(next_page) != 0:
        next_page_url = urljoin(base=url, url=next_page[0])
        scrape(url=next_page_url)

In [47]:
scrape(url="https://coinmarketcap.com")

In [49]:
all_currenceis[:2]

[{'_id': 1,
  'name': 'Bitcoin',
  'market cap': '2.10868810294e+11',
  'price': '11806.1528333',
  'change(24h)': '0.743107'},
 {'_id': 2,
  'name': 'Ethereum',
  'market cap': '24783346505.8',
  'price': '231.172455837',
  'change(24h)': '-0.927494'}]

In [53]:
import requests
from lxml import html
from urllib.parse import urljoin

top_movies = []

resp = requests.get(url='https://www.imdb.com/search/title?genres=drama&groups=top_250&sort=user_rating,desc', headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
})

tree = html.fromstring(html=resp.text)
movies = tree.xpath("//div[@class='lister-item-content']")

In [54]:
for movie in movies:
    m = {
        'title': movie.xpath(".//h3/a/text()")[0],
        'year of release': movie.xpath(".//h3/span[contains(@class, 'lister-item-year')]/text()")[0],
        'duration': movie.xpath(".//p/span[@class='runtime']/text()")[0],
        'rating': movie.xpath(".//div[@class='ratings-bar']/div[@name='ir']/@data-value")[0]
    }
    top_movies.append(m)
    
print(top_movies[:2])

[{'title': 'The Shawshank Redemption', 'year of release': '(1994)', 'duration': '142 min', 'rating': '9.3'}, {'title': 'The Godfather', 'year of release': '(1972)', 'duration': '175 min', 'rating': '9.2'}]


In [55]:
# with recursion

top_movies = []


def scrape(pageURL):
    resp = requests.get(url=pageURL, headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
    })

    tree = html.fromstring(html=resp.text)
    movies = tree.xpath("//div[@class='lister-item-content']")

    for movie in movies:
        m = {
            'title': movie.xpath(".//h3/a/text()")[0],
            'year of release': movie.xpath(".//h3/span[contains(@class, 'lister-item-year')]/text()")[0],
            'duration': movie.xpath(".//p/span[@class='runtime']/text()")[0],
            'rating': movie.xpath(".//div[@class='ratings-bar']/div[@name='ir']/@data-value")[0]
        }
        top_movies.append(m)

    next_page = tree.xpath("//div[@class='desc']/a[contains(text(), 'Next')]/@href")

    if len(next_page) != 0:
        scrape(pageURL=urljoin(base=pageURL, url=next_page[0]))
    else:
        return

In [57]:
scrape(pageURL='https://www.imdb.com/search/title?genres=drama&groups=top_250&sort=user_rating,desc')
print(top_movies[:2])

[{'title': 'The Shawshank Redemption', 'year of release': '(1994)', 'duration': '142 min', 'rating': '9.3'}, {'title': 'The Godfather', 'year of release': '(1972)', 'duration': '175 min', 'rating': '9.2'}]
